In [ ]:
from z3 import *  # Importa todas as funcionalidades da biblioteca Z3 para lógica e satisfatibilidade
import random  # Importa o módulo random para gerar números aleatórios

# Definir as funções dos LFSRs conforme a cifra A5/1
def step_lfsr1(lfsr):
    # Calcula o feedback para o LFSR1 baseado na lógica da cifra A5/1
    feedback = (lfsr >> 18) ^ (lfsr >> 17) ^ (lfsr >> 16) ^ (lfsr >> 13)  # Obtém o feedback dos bits relevantes
    return LShR(lfsr, 1) | (feedback << 18)  # Faz um deslocamento à direita e insere o feedback

def step_lfsr2(lfsr):
    # Calcula o feedback para o LFSR2
    feedback = (lfsr >> 21) ^ (lfsr >> 20)  # Obtém o feedback dos bits relevantes
    return LShR(lfsr, 1) | (feedback << 21)  # Faz um deslocamento à direita e insere o feedback

def step_lfsr3(lfsr):
    # Calcula o feedback para o LFSR3
    feedback = (lfsr >> 22) ^ (lfsr >> 21) ^ (lfsr >> 20) ^ (lfsr >> 7)  # Obtém o feedback dos bits relevantes
    return LShR(lfsr, 1) | (feedback << 22)  # Faz um deslocamento à direita e insere o feedback

def majority_bit(x, y, z):
    # Calcula o bit majoritário entre três bits
    # Soma os bits e verifica se a soma é maior ou igual a 2 para determinar o bit majoritário
    return If((Extract(0, 0, x) + Extract(0, 0, y) + Extract(0, 0, z)) >= 2, 1, 0)

# Função para simular a saída do gerador de chaves A5/1
def simulate_a5_1(lfsr1_initial, lfsr2_initial, lfsr3_initial, steps):
    lfsr1 = lfsr1_initial  # Inicializa LFSR1 com o valor inicial
    lfsr2 = lfsr2_initial  # Inicializa LFSR2 com o valor inicial
    lfsr3 = lfsr3_initial  # Inicializa LFSR3 com o valor inicial
    output_stream = []  # Cria uma lista para armazenar a sequência de saída

    for _ in range(steps):  # Itera pelo número de passos especificado
        majority = majority_bit(lfsr1, lfsr2, lfsr3)  # Calcula o bit majoritário com os três LFSRs
        output_stream.append(majority)  # Adiciona o bit majoritário à sequência de saída

        # Adiciona a condição correta para a comparação
        if majority == 1:  # Se o bit majoritário for 1
            lfsr1 = step_lfsr1(lfsr1)  # Atualiza LFSR1
            lfsr2 = step_lfsr2(lfsr2)  # Atualiza LFSR2
            lfsr3 = step_lfsr3(lfsr3)  # Atualiza LFSR3

    return output_stream  # Retorna a sequência de saída gerada

def detect_burst_of_zeros(solver, output_stream, t):
    # Detecta sequências de zeros na sequência de saída
    for i in range(len(output_stream) - t + 1):  # Percorre a sequência de saída
        # Verifica se existe uma sequência de t zeros consecutivos
        zero_burst = And([output_stream[i + j] == 0 for j in range(t)])  # Cria a condição para t zeros
        solver.add(zero_burst)  # Adiciona a condição ao solver

def detect_repeated_burst(solver, output_stream, t):
    # Detecta bursts repetidos na sequência de saída
    for i in range(len(output_stream) - t + 1):  # Percorre a sequência de saída
        for j in range(i + 1, len(output_stream) - t + 1):  # Compara bursts a partir do índice i + 1
            burst1 = output_stream[i:i+t]  # Primeiro burst a partir do índice i
            burst2 = output_stream[j:j+t]  # Segundo burst a partir do índice j
            solver.add(burst1 == burst2)  # Adiciona a condição de que os bursts devem ser iguais

def run_a5_1_simulation(t, steps):
    # Criar um solver
    solver = Solver()  # Cria uma nova instância do solver

    # Gerar estados iniciais aleatórios para os LFSRs
    lfsr1_initial = BitVecVal(random.randint(0, 2**19 - 1), 19)  # LFSR1 com um valor inicial aleatório
    lfsr2_initial = BitVecVal(random.randint(0, 2**22 - 1), 22)  # LFSR2 com um valor inicial aleatório
    lfsr3_initial = BitVecVal(random.randint(0, 2**23 - 1), 23)  # LFSR3 com um valor inicial aleatório

    # Gerar a sequência de saída com base nesses estados
    output_stream = simulate_a5_1(lfsr1_initial, lfsr2_initial, lfsr3_initial, steps)

    # Verificar a propriedade de burst de zeros
    detect_burst_of_zeros(solver, output_stream, t)

    # Verificar a propriedade de repetição de burst
    detect_repeated_burst(solver, output_stream, t)

    # Tentar satisfazer as restrições e verificar se a propriedade ocorre
    if solver.check() == sat:  # Verifica se as condições adicionadas são satisfatíveis
        print("Propriedades atingíveis a partir do estado inicial aleatório!")  # Se satisfatível, imprime mensagem
    else:
        print("Propriedades não atingíveis a partir do estado inicial aleatório.")  # Se não, imprime mensagem

# Exemplo de execução
t = 3  # Tamanho do burst (número de zeros consecutivos a detectar)
steps = 16  # Número de passos na simulação (quantidade de bits a gerar)
run_a5_1_simulation(t, steps)  # Executa a simulação com os parâmetros definidos



Propriedades atingíveis a partir do estado inicial aleatório!
